# This notebook provides portfolio weights based on the historical data to the backtesting notebook

In [1]:
# !pip uninstall deepdowmine
#!pip install git+https://github.com/dsman1823/deepdowmine.git
#!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch # <---- to Enable SVD 

In [2]:
import torch

import pandas as pd
import numpy as np

from datetime import datetime 
from deepdowmine.nn import LinearNetMine


In [3]:
RETS_FILE_PATH = 'historical_returns.csv'



loockback, gap, horizon = 50, 0, 5
n_assets = 5


## Load NN from dict

In [4]:
LINEAR_NET_OSHARPE_FILE_PATH = r'./NNs/linear_net_50x5_sharpe.pth' # LinearNetMine with ordinary Sharpe cost

In [26]:
# years parameter reprsenets amount of years after the backtest stasrt: the model dowloaded is the the model trained 
# on the period (train_start, back_test_start + years) 


# LinearNetMine with ordinary sharpe cost function
def load_linear_net_osharpe(years=0):
    network = LinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{years}.pth'))
    print(fr'./NNs/linear_net_50x5_sharpe_{years}.pth')
    return network.eval()
    

## Core functions

In [6]:
def transform_rets_to_NN_input(rets):
    # tranform (loockback, n_assets) df into (1, 1, loockback, n_assets)=(sample_size, n_channels, loockback, n_assets) tensor 
    returns_np = rets.to_numpy()

    # Add the required dimensions: (n_samples, n_channels, lookback, n_assets)
    returns_np_expanded = np.expand_dims(returns_np, axis=0)  # Adds n_samples dimension
    returns_np_expanded = np.expand_dims(returns_np_expanded, axis=0)  
    return torch.from_numpy(returns_np_expanded).float()

def weights_from_NN(rets, network):
    X = transform_rets_to_NN_input(rets)
    return network(X).detach().numpy()[0]

# method is a function, used for obtaining the returns
def get_weights(method):
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    return method(rets)

In [7]:
rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
X = transform_rets_to_NN_input(rets)
network = LinearNetMine(1, loockback, n_assets, p=0.5)
network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{2}.pth'))
network.eval()(X).detach().numpy()[0]

array([ 0.0641624 ,  0.25815332, -0.01579298,  0.6089077 ,  0.08456962],
      dtype=float32)

## NNs and related methods


### NNs

networks[i] method is trained on (train_start, backtest_start + i years) data interval

In [40]:
years = [0, 1, 2]#, 1, 2]

linear_osharpe_networks = [load_linear_net_osharpe(years=i) for i in years]

./NNs/linear_net_50x5_sharpe_0.pth
./NNs/linear_net_50x5_sharpe_1.pth
./NNs/linear_net_50x5_sharpe_2.pth


### methods

In [41]:
linear_osharpe_networks[1].linear0.bias[0:3]

tensor([ 0.0128,  0.0384, -0.0396], grad_fn=<SliceBackward0>)

In [42]:
linear_osharpe_networks[0].linear0.bias[0:3]

tensor([0.0425, 0.0468, 0.0411], grad_fn=<SliceBackward0>)

In [43]:
# note the format of lambda 
linear_osharpe_methods = [lambda r, i=i: weights_from_NN(r, linear_osharpe_networks[i]) for i in years] 

In [44]:
[get_weights(linear_osharpe_methods[i]) for i in years]

[array([-0.32369575,  1.4470922 , -0.1738278 ,  0.27279902, -0.2223676 ],
       dtype=float32),
 array([-0.03426138,  0.46759862, -0.14483896,  0.78433096, -0.07282931],
       dtype=float32),
 array([ 0.0641624 ,  0.25815332, -0.01579298,  0.6089077 ,  0.08456962],
       dtype=float32)]

# Server

In [45]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, date

RETRAINING_DATE1 = date(2021, 12, 15)
RETRAINING_DATE2 = date(2022, 12, 15)

In [46]:
from flask import Flask, request, jsonify
from flask_cors import CORS 

import numpy as np

app = Flask(__name__)
CORS(app)  # Enable CORS for your Flask app


@app.route('/get_weights', methods=['GET'])
def get_request():
    # Extract date string from the query parameters
    date_str = request.args.get('date', None)
    
    # Simple validation to check if date is provided
    if not date_str:
        return jsonify({"error": "Missing date parameter"}), 400

    # Try to convert the date string to a datetime object
    try:
        # Note that now we're only parsing the date, not the time
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        # If there is an error in parsing the date, return an error message
        return jsonify({"error": "Invalid date format. Please use YYYY-MM-DD format."}), 400
    
#     years_plus = 0
#     if RETRAINING_DATE1 < date < RETRAINING_DATE2:
#         years_plus = 1
#         print(f'Working with model: {years_plus}')
#     if RETRAINING_DATE2 < date:
#         years_plus = 2
#         print(f'Working with model: {years_plus}')
    
#     weights = get_weights(linear_osharpe_methods[years_plus]).tolist()
    
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    X = transform_rets_to_NN_input(rets)
    network = LinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{0}.pth'))
    
    if date < RETRAINING_DATE1:
        print(0)
    if RETRAINING_DATE1 < date < RETRAINING_DATE2:
        print('1')
        network = LinearNetMine(1, loockback, n_assets, p=0.5)
        network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{1}.pth'))
    if RETRAINING_DATE2 < date:
        network = LinearNetMine(1, loockback, n_assets, p=0.5)
        network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{2}.pth'))
        print('2')
    
    weights = network.eval()(X).detach().numpy()[0].tolist()
    
        
    #weights = weights_from_NN(rets, linear_osharpe_networks[0]).tolist()
    #print(weights)
    # Return the vector as JSON, using the string representation of the date for simplicity
    return jsonify({'weights': weights})


In [50]:
rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
X = transform_rets_to_NN_input(rets)
network = LinearNetMine(1, loockback, n_assets, p=0.5)
network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{0}.pth'))
weights = network.eval()(X).detach().numpy()[0].tolist()
weights

[-0.32369574904441833,
 1.4470921754837036,
 -0.17382779717445374,
 0.272799015045166,
 -0.22236759960651398]

In [ ]:
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Mar/2024 13:05:43] "GET /get_weights?date=2020-12-21 HTTP/1.1" 200 -


0


127.0.0.1 - - [06/Mar/2024 13:05:45] "GET /get_weights?date=2020-12-28 HTTP/1.1" 200 -


0


...

2


127.0.0.1 - - [06/Mar/2024 14:03:01] "GET /get_weights?date=2023-12-04 HTTP/1.1" 200 -


2


127.0.0.1 - - [06/Mar/2024 14:03:03] "GET /get_weights?date=2023-12-11 HTTP/1.1" 200 -


2


In [ ]:
update the 